In [1]:
import csv
from decimal import Decimal



In [2]:

def loadInvestments(investmentFilename):
    investments = []

    with open(investmentFilename, 'r') as file:
        reader = csv.DictReader(file)

        # To skip the first row
        next(reader)
        

        for row in reader:
            name = row['RegionName']
            cost = Decimal(row['Zhvi']).quantize(Decimal("0.00"))
            roi = cost * Decimal(row['10Year'])
                   
            investments.append((name, cost, roi))

    return investments


# Test

investmentFilename = 'state_zhvi_summary_allhomes.csv'
investments = loadInvestments(investmentFilename)


for investment in investments[:5]:
    print(f"Investment Name: {investment[0]}, InvestmentCost: ${
          investment[1]}, Estimated ROI: ${investment[2]}")

Investment Name: California, InvestmentCost: $547700.00, Estimated ROI: $26325.19830920000
Investment Name: Texas, InvestmentCost: $196100.00, Estimated ROI: $8026.26083080000
Investment Name: New York, InvestmentCost: $300400.00, Estimated ROI: $3974.55545080000
Investment Name: Florida, InvestmentCost: $233200.00, Estimated ROI: $9707.10671040000
Investment Name: Illinois, InvestmentCost: $181100.00, Estimated ROI: $914.91647560000


In [3]:
def optimizeInvestments(investments, budget):
    n = len(investments)

    # Initialize DP table and traceback table
    dp = [[Decimal("0.00") for _ in range(budget + 1)] for _ in range(n + 1)]
    traceback = [[False for _ in range(budget + 1)] for _ in range(n + 1)]

    # Fill in the DP table
    for i in range(1, n + 1):
        name, cost, roi = investments[i - 1]
        cost = int(cost)  # Convert to integer for indexing

        for j in range(budget + 1):
            if cost <= j:
                # Option to include the investment
                include_investment = roi + dp[i - 1][j - cost]
                exclude_investment = dp[i - 1][j]

                if include_investment > exclude_investment:
                    dp[i][j] = include_investment
                    # Mark that we included this investment
                    traceback[i][j] = True
                else:
                    dp[i][j] = exclude_investment
            else:
                dp[i][j] = dp[i - 1][j]  # Can't include, so carry forward

    # Optimal ROI
    optimal_roi = dp[n][budget]

    # Traceback to find selected investments
    selected_investments = []
    remaining_budget = budget

    for i in range(n, 0, -1):
        if traceback[i][remaining_budget]:
            name, cost, roi = investments[i - 1]
            selected_investments.append(name)
            remaining_budget -= int(cost)

    # Reverse the selected investments to maintain the original order
    selected_investments.reverse()

    return optimal_roi, selected_investments

In [4]:

# Define the budget
budget = 1000000

# Run the optimizeInvestments function with the loaded data
optimal_roi, selected_investments = optimizeInvestments(investments, budget)

# Print  results
print(f"Optimal Return on Investment: ${optimal_roi}")
print("Selected Investments:", selected_investments)

Optimal Return on Investment: $49455.55658990000
Selected Investments: ['Michigan', 'Tennessee', 'Colorado', 'Nevada']
